In [3]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [4]:
training_df = pd.read_csv("../train.csv")

In [5]:
training_df.shape

(800, 248)

In [6]:
training_df.set_index("Timestamp", inplace=True)

In [7]:
training_df.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Country,Afghanistan,Albania,Algeria,American Samoa,Andorra,...,Zambia,Zimbabwe,afternoon,evening,morning,night,Ad Topic Line,Ad Line Length,Ad Line Word Length,Clicked on Ad
Timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-29 00:45:19,-1.003266,-0.475928,-1.074513,-1.369008,Fiji,0,0,0,0,0,...,0,0,0,0,0,1,Optimized intermediate help-desk,-0.247760,-0.451417,1
2016-01-05 20:58:42,0.838468,0.211953,0.176047,1.637189,Liberia,0,0,0,0,0,...,0,0,0,1,0,0,Networked regional Local Area Network,0.629266,3.723539,0
2016-06-18 16:02:34,-1.895712,0.441247,-0.705899,-0.666237,Mongolia,0,0,0,0,0,...,0,0,1,0,0,0,Fully-configurable context-sensitive Graphic I...,3.611156,1.636061,1
2016-06-21 00:52:47,0.386877,-0.705222,1.065619,1.014610,France,0,0,0,0,0,...,0,0,0,0,0,1,Assimilated stable encryption,-0.773976,-0.451417,0
2016-04-17 05:08:52,1.199110,1.358422,-0.643565,-1.009855,Kyrgyz Republic,0,0,0,0,0,...,0,0,0,0,1,0,Inverse discrete extranet,-1.475597,-0.451417,1


In [8]:
training_X_cols = list(training_df.select_dtypes(exclude=["object"]).columns)
training_X_cols.remove('Clicked on Ad')
training_y_cols = 'Clicked on Ad'

In [9]:
param_grid = [
    {"C":[1,10,100],"kernel":["linear","poly","rbf"],"degree":[3,6,9]},
]

In [10]:
grid_search = GridSearchCV(SVC(),param_grid,cv=5, scoring=["neg_mean_squared_error","accuracy"],refit="accuracy")

In [11]:
grid_search.fit(training_df[training_X_cols],training_df[training_y_cols])

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100], 'degree': [3, 6, 9],
                          'kernel': ['linear', 'poly', 'rbf']}],
             refit='accuracy', scoring=['neg_mean_squared_error', 'accuracy'])

In [12]:
best_estimator = grid_search.best_estimator_
print(best_estimator)

SVC(C=10, kernel='linear')


In [13]:
results = grid_search.cv_results_
for key in results.keys():
    print(key)
    print(results[key])
    print("\n\n")

mean_fit_time
[0.02539654 0.03930736 0.02990556 0.02135253 0.0826262  0.03032713
 0.0209445  0.08256392 0.02852416 0.02394695 0.03489742 0.03269734
 0.02392659 0.09516025 0.03053427 0.02254043 0.09495397 0.03313613
 0.03290429 0.034905   0.03229365 0.03412566 0.09893265 0.03331008
 0.03648825 0.10372853 0.03350325]



std_fit_time
[0.00455725 0.00247837 0.00063062 0.00148083 0.00231949 0.00135027
 0.00088915 0.0016996  0.00079817 0.00245977 0.0008928  0.00312386
 0.00251242 0.00242509 0.00048715 0.00162662 0.00278599 0.00038888
 0.0082087  0.00209588 0.00048779 0.00793809 0.00825333 0.00050666
 0.0088443  0.00418308 0.00101737]



mean_score_time
[0.00617008 0.01098413 0.00817842 0.00579886 0.02152109 0.00797939
 0.00558438 0.0207509  0.00837703 0.00538583 0.00918961 0.00878763
 0.00578465 0.01994658 0.00797668 0.00498576 0.02073636 0.00954957
 0.00479488 0.00958409 0.00879421 0.00480285 0.02093844 0.00917659
 0.00541372 0.02115531 0.00860662]



std_score_time
[3.92477751e-04 1.083001

In [14]:
for mean_squared_error, accuracy, params in zip(results["mean_test_neg_mean_squared_error"], results["mean_test_accuracy"], results["params"]):
    print(f"params: {params} mse score: {-mean_squared_error} rmse: {np.sqrt(-mean_squared_error)} accuracy: {accuracy} ")
    print("\n")

params: {'C': 1, 'degree': 3, 'kernel': 'linear'} mse score: 0.0375 rmse: 0.19364916731037085 accuracy: 0.9625 


params: {'C': 1, 'degree': 3, 'kernel': 'poly'} mse score: 0.045 rmse: 0.21213203435596426 accuracy: 0.9550000000000001 


params: {'C': 1, 'degree': 3, 'kernel': 'rbf'} mse score: 0.04125 rmse: 0.203100960115899 accuracy: 0.95875 


params: {'C': 1, 'degree': 6, 'kernel': 'linear'} mse score: 0.0375 rmse: 0.19364916731037085 accuracy: 0.9625 


params: {'C': 1, 'degree': 6, 'kernel': 'poly'} mse score: 0.17750000000000002 rmse: 0.42130748865881795 accuracy: 0.8225000000000001 


params: {'C': 1, 'degree': 6, 'kernel': 'rbf'} mse score: 0.04125 rmse: 0.203100960115899 accuracy: 0.95875 


params: {'C': 1, 'degree': 9, 'kernel': 'linear'} mse score: 0.0375 rmse: 0.19364916731037085 accuracy: 0.9625 


params: {'C': 1, 'degree': 9, 'kernel': 'poly'} mse score: 0.28625 rmse: 0.5350233639758174 accuracy: 0.71375 


params: {'C': 1, 'degree': 9, 'kernel': 'rbf'} mse score: 0.041